In [1]:
import numpy as np
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchvision
import torch.utils.data as dt
import torchvision.transforms as transforms
import time
import os
import random
from PIL import Image
from youtube_scraper import download_csv_audio
from preprocessor import autoencoder_hilbert_data, autoencoder_spectrogram_data
from dataset import AutoDataset

In [2]:
og = os.getcwd()

In [3]:
def get_autoencoder_data(csv_path, win_len = 2048, hilbert = True, set_percent = 0.1):
    download_csv_audio(csv_path)                                                                    #Download all the youtube links as .wav files
    os.chdir(og+"/Downloads/Audios/")
    n = 0
    counter = 0
    for file in os.listdir(og+'/Downloads/Audios'):
        n += 1                                                                                          #where the youtube files were saved
        print(f"Iteration{n}")
        print(file)
        if file.endswith(".wav"):                                                                   #For each .wav file in the downloaded path
            print("executed1")
            if hilbert: 
                newpath = og + '/Data/Hilbert/Autoencoder/'                                                 #If hilbert is required, save the .wav files as hilbert curve data
                print("Executed2")
                data = autoencoder_hilbert_data(file, win_len = win_len)
            else:                                                                                                                #Else, save the .wav files as spectrogram data
                newpath = og + 'Data/Spectrogram/Autoencoder/'
                print("Executed 3")                                                                                       
                data = autoencoder_spectrogram_data(file, win_len = win_len)
            
            print('executed4')
        
            #print(data)
            try:
                for array in data:
                    counter += 1
                    norm = (array - np.min(array))/(np.max(array) - np.min(array))*255
                    norm = norm.astype(np.uint8)
                    im = Image.fromarray(norm)
                    im.save(newpath + str(counter) + ".png")
            except:
                continue
    
    os.chdir(os.path.join(og,"/Downloads/Audios/"))
    autoencoder_dataset(newpath, set_percent = 0.1)
    
    

In [4]:
def autoencoder_dataset(path, set_percent = 0.1):
    transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.5), (0.5/3))])   #Make sure images are tensors, lie within 0 and 1
    dataset = AutoDataset(path, transform = transform) #need to implement   #Turn the images into a dataset
    set_size = int( set_percent * len( dataset ) )                                                        #Length of test and validation sets
    return dt.random_split(dataset, [len( dataset ) - 2 * set_size, set_size, set_size])                #Return datasets in order of training set, validation set, test setde

In [5]:
os.chdir(og)
train, val, test = get_autoencoder_data('piece.csv', win_len=65536, hilbert = False, set_percent = 0.1)

[youtube] Bh_nHgbhYRg: Downloading webpage
[download] Antonìn Dvořák - Symphony No. 9 'From the New World' Op. 95 _ Claudio Abbado. (Live Performance).mp4 has already been downloaded
[download] 100% of 74.49MiB
[youtube] Bh_nHgbhYRg: Downloading webpage
MoviePy - Writing audio in Antonìn Dvořák - Symphony No. 9 'From the New World' Op. 95 _ Claudio Abbado. (Live Performance).wav


MoviePy - Done.
Iteration1
Antonìn Dvořák - Symphony No. 9 'From the New World' Op. 95 _ Claudio Abbado. (Live Performance).wav
executed1
Executed 3


TypeError: slice indices must be integers or None or have an __index__ method

In [ ]:
print(train.dataset.root)

Data\Hilbert\Autoencoder\Elgar


In [ ]:
download_csv_audio('piece.csv')

[youtube] Bh_nHgbhYRg: Downloading webpage
[download] Antonìn Dvořák - Symphony No. 9 'From the New World' Op. 95 _ Claudio Abbado. (Live Performance).mp4 has already been downloaded
[download] 100% of 74.49MiB
[youtube] Bh_nHgbhYRg: Downloading webpage
MoviePy - Writing audio in Antonìn Dvořák - Symphony No. 9 'From the New World' Op. 95 _ Claudio Abbado. (Live Performance).wav


MoviePy - Done.


In [ ]:
os.chdir(og)
print(os.getcwd())
loader = torch.utils.data.DataLoader(train, batch_size=64, num_workers=0)
for i, sample in enumerate(loader):
    if i > 2:
        break
    print(sample)

c:\Users\Adai9\GitHub\Lingling-Bot
tensor([[[[-3.0000, -3.0000, -3.0000,  ..., -0.7412, -0.6000, -1.5176],
          [-3.0000, -3.0000, -3.0000,  ..., -0.7647, -1.5882, -0.4118],
          [-3.0000, -3.0000, -2.8118,  ..., -0.6471, -0.4824, -1.1176],
          ...,
          [-3.0000, -3.0000, -3.0000,  ..., -3.0000, -3.0000, -3.0000],
          [-3.0000, -3.0000, -3.0000,  ..., -3.0000, -3.0000, -2.8353],
          [-3.0000, -3.0000, -3.0000,  ..., -3.0000, -3.0000, -3.0000]],

         [[-3.0000, -3.0000, -3.0000,  ..., -0.7412, -0.6000, -1.5176],
          [-3.0000, -3.0000, -3.0000,  ..., -0.7647, -1.5882, -0.4118],
          [-3.0000, -3.0000, -2.8118,  ..., -0.6471, -0.4824, -1.1176],
          ...,
          [-3.0000, -3.0000, -3.0000,  ..., -3.0000, -3.0000, -3.0000],
          [-3.0000, -3.0000, -3.0000,  ..., -3.0000, -3.0000, -2.8353],
          [-3.0000, -3.0000, -3.0000,  ..., -3.0000, -3.0000, -3.0000]],

         [[-3.0000, -3.0000, -3.0000,  ..., -0.7412, -0.6000, -1.51